# Pandas Tips & Tricks

This notebook presents various tricks to manipulate your data, which are typically non-obvious to a novice in Pandas and data science.

In [1]:
%matplotlib notebook

import time
from IPython.display import HTML, clear_output
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Reading Data

### Combining Multiple CSV Files

This shows how to create a single dataframe from multiple files that share the same structure (columns).

In [2]:
import glob

files = '../data/commits*.tsv'
df = pd.concat([pd.read_csv(x, sep='\t', parse_dates=['Date']) for x in glob.glob(files)], 
               ignore_index=True)

!wc -l {files}
print('♯', len(df))

  11 ../data/commits1.tsv
  12 ../data/commits2.tsv
  23 total
♯ 21


In [3]:
len(df.Author.unique()), len(pd.unique(df.Author))

(2, 2)

## Inspecting Dataframes

Looking at the contents and metadata of your dataframes is quite important, to better understand the data they represent and then successfully transform it into the results you need.

In [4]:
# Data dimensions (rows, cols)
df.shape

(21, 4)

In [5]:
# Data types
df.dtypes

SHA                                      object
Author                                   object
Date       datetime64[ns, pytz.FixedOffset(60)]
Message                                  object
dtype: object

If you look at a sample, it is often useful to transpose the data, especially when you have many columns.

In [6]:
print(df.head(2).transpose())

                                           0                            1
SHA                                  8fe0ea9                      9de3457
Author                              jhermann                     jhermann
Date               2019-02-16 06:04:19+01:00    2019-02-16 05:57:50+01:00
Message  :link: Python Data Science Handbook  add requirements for Binder


And then there is `describe` with some core statistics about the dataframe…

In [7]:
print(df.describe().transpose())

        count unique                                              top freq  \
SHA        21     21                                          6454187    1   
Author     21      2                                         jhermann   20   
Date       21     21                        2019-02-15 21:23:05+01:00    1   
Message    21     21  link to Netflix 'Notebook Innovation' blog post    1   

                             first                       last  
SHA                            NaN                        NaN  
Author                         NaN                        NaN  
Date     2019-02-11 16:06:17+01:00  2019-02-16 06:04:19+01:00  
Message                        NaN                        NaN  


… and `info` with more technical information.

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 4 columns):
SHA        21 non-null object
Author     21 non-null object
Date       21 non-null datetime64[ns, pytz.FixedOffset(60)]
Message    21 non-null object
dtypes: datetime64[ns, pytz.FixedOffset(60)](1), object(3)
memory usage: 752.0+ bytes


## Writing Results
### Writing Spreadsheet Files
*TODO*

## Filtering Rows

You can use [loc](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.loc.html) in combination with a `bool` array to select a subset of rows. That array is conveniently created by applying conditions to columns.

The first example uses regex matching…

In [9]:
print(df.loc[df.Message.str.match('altair', case=False)]
             .reindex(['Date', 'Message'], axis=1))

                        Date                                   Message
7  2019-02-15 17:17:30+01:00               Altair: use non-random data
8  2019-02-15 16:58:26+01:00  Altair: Publishing Charts with nbconvert
10 2019-02-15 10:40:04+01:00                      Altair setup details


Another option is using simple comparison operators, e.g. `!=` like here…

In [10]:
print(df.loc[df.Author != 'jhermann']
             .reindex(['Date', 'Message'], axis=1))

                        Date         Message
20 2019-02-11 16:06:17+01:00  Initial commit


Note that the condition creates a `bool` array, that then is taken by `loc[…]` to select the matching rows.

In [11]:
list(df.Author.iloc[-5:] != 'jhermann')

[False, False, False, False, True]

## Manipulating Columns

### Adding or Replacing Columns
Changing the values of a column or adding a whole new one can be done by actual assignment or by calling `assign`.

In [12]:
morecols = df.assign(Words=df.Message.str.split().apply(len))
print('Column ♯:', len(df.columns), 'vs.', len(morecols.columns))
print(morecols.head(2).iloc[:, -3:])

Column ♯: 4 vs. 5
                       Date                              Message  Words
0 2019-02-16 06:04:19+01:00  :link: Python Data Science Handbook      5
1 2019-02-16 05:57:50+01:00          add requirements for Binder      4


Using assigment is inplace and changes the dataframe.

In [13]:
morecols['Zero'] = 0
print(morecols.head(2).iloc[:, -4:])

                       Date                              Message  Words  Zero
0 2019-02-16 06:04:19+01:00  :link: Python Data Science Handbook      5     0
1 2019-02-16 05:57:50+01:00          add requirements for Binder      4     0


### Renaming Columns
This one's easy, just call [rename](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rename.html). Columns can be specified in various formats, like a mapping from old to new. Renaming can also be done inplace, the default is to copy.

In [14]:
print(morecols.rename(columns=dict(Message='Text')).head(1).iloc[:, -4:])

                       Date                                 Text  Words  Zero
0 2019-02-16 06:04:19+01:00  :link: Python Data Science Handbook      5     0


To rename all columns, just `zip` the existing names with the new ones.

In [15]:
numcols = morecols.rename(
    columns=dict(zip(morecols.columns, 
                     range(len(morecols.columns)))))
print(numcols.head(1).transpose())

                                     0
0                              8fe0ea9
1                             jhermann
2            2019-02-16 06:04:19+01:00
3  :link: Python Data Science Handbook
4                                    5
5                                    0


To rename according to some logic, like a regex substitution or similar, provide a mapper function.

In [16]:
print(morecols.rename(mapper=str.upper, axis=1).head(1).transpose())

                                           0
SHA                                  8fe0ea9
AUTHOR                              jhermann
DATE               2019-02-16 06:04:19+01:00
MESSAGE  :link: Python Data Science Handbook
WORDS                                      5
ZERO                                       0


### Deleting Columns
*TODO*

### Selecting Columns

In [17]:
print(df[['Date', 'Message']].head(1))

                       Date                              Message
0 2019-02-16 06:04:19+01:00  :link: Python Data Science Handbook


## String Manipulation

The new `Day` column is just the first word out of the `Date` column. By splitting with `expand=True` two columns are created (instead of one column with tuples), so we can select the first column only and add this to the dataframe.

In [18]:
df = df.assign(Day=df.Date.astype(str).str.split(n=1, expand=True)[0])
print(df.head(1).iloc[:, -3:])

                       Date                              Message         Day
0 2019-02-16 06:04:19+01:00  :link: Python Data Science Handbook  2019-02-16


Since `Date` is a `datetime64` column, we can also use the [DatetimeProperties](http://pandas.pydata.org/pandas-docs/version/0.15.0/api.html#datetimelike-properties) accessor for day extraction.

In [19]:
df = df.assign(Day=df.Date.dt.date)
print(df.head(1).iloc[:, -3:])

                       Date                              Message         Day
0 2019-02-16 06:04:19+01:00  :link: Python Data Science Handbook  2019-02-16


## Counting

To visualize data in bar or other magnitude charts, you have to count subsets of your raw data.

In [20]:
commits_per_day = df.Day.value_counts().to_frame().sort_index()
_ = commits_per_day.plot.barh(legend=False, figsize=(5, 2))

chart_img = 'img/pandas-barh.png'
plt.savefig(chart_img)
clear_output()
HTML('<img src=\"{}?{}\"></img>'.format(chart_img, time.time()))

## Aggregation

Grouping values by one or more columns and then applying an operation to fold those values into a single scalar.

In [21]:
letters = list("Pandas")
codes = pd.DataFrame(dict(Letter=letters, Code=list(map(ord, letters))))
codes = codes.groupby('Letter').aggregate(np.sum)
print(codes.transpose())

Letter   P    a    d    n    s
Code    80  194  100  110  115


Using `reset_index` moves the grouping column(s) from the index to ordinary columns.

In [22]:
codes = codes.reset_index()
print(codes.transpose())

         0    1    2    3    4
Letter   P    a    d    n    s
Code    80  194  100  110  115
